In [158]:
import tensorflow as tf
import numpy as np
import sys
import pandas as pd
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching
from branchingdnn.utils import * 
from branchingdnn.profiler import model_profiler as profiler
import h5py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Concatenate
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

In [54]:
###normal
inputs = keras.Input(shape=(2,1))
x = keras.layers.Dense(256)(inputs)
x = keras.layers.Dense(256)(x)
x = keras.layers.Dense(256)(x)

softmax = keras.layers.Dense(10, activation='softmax')(x)


In [55]:
# model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
model = keras.Model(inputs=inputs, outputs=[softmax], name="bandit")
#     model.save("models/alexNetv6_compressed.hdf5")
model.summary()

Model: "bandit"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 2, 1)]            0         
_________________________________________________________________
dense_32 (Dense)             (None, 2, 256)            512       
_________________________________________________________________
dense_33 (Dense)             (None, 2, 256)            65792     
_________________________________________________________________
dense_34 (Dense)             (None, 2, 256)            65792     
_________________________________________________________________
dense_35 (Dense)             (None, 2, 10)             2570      
Total params: 134,666
Trainable params: 134,666
Non-trainable params: 0
_________________________________________________________________


In [178]:
dataset = pd.read_csv("bandit_csv.csv")
print(dataset.head())
bandit_features = dataset.copy()
bandit_features.pop("Labels")
bandit_labels = bandit_features.pop("Binary_Labels")

test_size = 1000

bandit_test_features, bandit_test_labels = bandit_features[:test_size], bandit_labels[:test_size] #get the first 5k training samples as validation set
bandit_features, bandit_labels = bandit_features[test_size:], bandit_labels[test_size:]
print(len(bandit_features))
print(len(bandit_labels))


print(len(bandit_test_features))
print(len(bandit_test_labels))

bandit_features = np.array(bandit_features)
bandit_features = bandit_features[~np.isnan(bandit_features).any(axis=1)]

bandit_features


   Prediction _compress  Labels   Entropy  Binary_Labels
0                     3       3  0.316679              1
1                     8       8  0.923481              1
2                     8       8  0.602183              1
3                     8       0  0.638663              0
4                     6       6  0.198454              1
9000
9000
1000
1000


array([[5.        , 0.19856652],
       [0.        , 1.03368626],
       [1.        , 1.03582609],
       ...,
       [5.        , 0.86906452],
       [1.        , 0.03519756],
       [7.        , 0.00886809]])

In [150]:
bandit_labels.head()

1000    1
1001    1
1002    0
1003    1
1004    1
Name: Binary_Labels, dtype: int64

In [206]:
bandit_model = tf.keras.Sequential([
  layers.Dense(256,activation='relu'),
  layers.Dense(124,activation='relu'),
  layers.Dense(64,activation='relu'),
  layers.Dense(1,activation='sigmoid')
])

bandit_model.compile(loss =tf.keras.losses.BinaryCrossentropy(),
                      optimizer = tf.optimizers.Adam(),metrics=['accuracy'])

In [207]:
bandit_model.fit(bandit_features, bandit_labels, epochs=10)

Epoch 1/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4920 - accuracy: 0.7670
Epoch 2/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4207 - accuracy: 0.7945
Epoch 3/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4137 - accuracy: 0.7890
Epoch 4/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4117 - accuracy: 0.7941
Epoch 5/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4029 - accuracy: 0.7989
Epoch 6/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4113 - accuracy: 0.7969
Epoch 7/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4102 - accuracy: 0.7937
Epoch 8/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4075 - accuracy: 0.7904
Epoch 9/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4188 - accuracy: 0.7924
Epoch 10/10
282/282 [==============================] - 1s 3ms/step - loss: 0.4112 - accuracy: 0.7951

In [188]:
predictions = bandit_model.evaluate(bandit_test_features, bandit_test_labels)
print(predictions)

32/32 [==============================] - 0s 2ms/step - loss: 0.4085 - accuracy: 0.8030
[0.4084572196006775, 0.8029999732971191]


In [205]:
# print(bandit_test_features.iloc[3])
# print(bandit_test_labels.iloc[3])
item = bandit_test_features.iloc[3]
labels = bandit_test_labels.copy()
# predictions = bandit_model.predict_classes(bandit_test_features)
predictions = np.round(bandit_model.predict(bandit_test_features))
predictions = np.array(predictions).reshape(1,1000)
# print(np.array(bandit_test_labels))
# print(predictions[0])

pred = pd.DataFrame(predictions)
pred.head()

results = pd.DataFrame(bandit_test_features)
results['labels']=labels
results['pred_label']=predictions[0]
results



,Prediction _compress,Entropy,labels,pred_label
0,3,0.316679,1,1.0
1,8,0.923481,1,1.0
2,8,0.602183,1,1.0
3,8,0.638663,0,1.0
4,6,0.198454,1,1.0
5,6,0.092814,1,1.0
6,1,0.069604,1,1.0
7,2,1.196157,0,1.0
8,3,0.941219,1,1.0
9,1,0.167460,1,1.0
